In [49]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
wijken = None
# load all parquet files into a single geodataframe from results/filter2
for file in os.listdir('results/filter2/wijken'):
    if file.endswith('.parquet'):
        if 'wijken' in locals():
            wijken = pd.concat([wijken, gpd.read_parquet(f'results/filter2/wijken/{file}')])
        else:
            wijken = gpd.read_parquet(f'results/filter2/wijken/{file}')

wijken.sample(3)

,wijkcode,wijknaam,gemeentecode,gemeentenaam,IND_WBI,H2O,OAD,STED,BEV_DICHTH,AANT_INW,...,L1_c_Municipal and governmental services,L1_c_Clubs and schools,"L1_c_gyms, fitness and centers",L1_c_Public fields and pitches,L1_c_Fast food,L1_c_Bars and cafes,L1_c_Stands and takeaway,"L1_c_Bus, tram and metro",L1_c_Public transport,L1_c_Train
1512,WK059924,Rotterdam-Noord-West,GM0599,Rotterdam,1.0,NEE,534.0,4.0,-99999999.0,5.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,0.0,0.0
123,WK008081,Dorpen Zuid-West,GM0080,Leeuwarden,1.0,NEE,83.0,5.0,78.0,3920.0,...,0.0,36.0,2.0,3.0,1.0,6.0,0.0,41.0,0.0,0.0
883,WK0363AE,Burgwallen-Oude Zijde,GM0363,Amsterdam,2.0,NEE,7768.0,1.0,13078.0,4565.0,...,0.0,0.0,0.0,0.0,52.0,157.0,7.0,9.0,0.0,0.0


In [50]:
sample = wijken.sample(1)
sample = wijken[wijken['wijknaam'] == 'Rozenknopje']
sample

,wijkcode,wijknaam,gemeentecode,gemeentenaam,IND_WBI,H2O,OAD,STED,BEV_DICHTH,AANT_INW,...,L1_c_Municipal and governmental services,L1_c_Clubs and schools,"L1_c_gyms, fitness and centers",L1_c_Public fields and pitches,L1_c_Fast food,L1_c_Bars and cafes,L1_c_Stands and takeaway,"L1_c_Bus, tram and metro",L1_c_Public transport,L1_c_Train
1822,WK077271,Rozenknopje,GM0772,Eindhoven,1.0,NEE,3626.0,1.0,6306.0,7020.0,...,0.0,3.0,2.0,1.0,5.0,3.0,1.0,20.0,0.0,0.0


In [53]:
agecols = list(wijken.filter(regex="P_.*_JR$").columns)
features = ['BEV_DICHTH', 'L0_altieri', 'L1_altieri']
wijken = wijken[wijken['BEV_DICHTH'] > 0]  # remove rows with 0 population density
# normalize features
wijken[features] = wijken[features].apply(lambda x: (x - x.mean()) / x.std())
wijken[agecols] = wijken[agecols].apply(lambda x: x/100)
sample = wijken.sample(1)
sample = wijken[wijken['wijknaam'] == 'Rozenknopje']

features = features + agecols
# calculate the distance between the sample and all other wijken, don't use the index
wijken['distance'] = np.linalg.norm(wijken[features] - sample[features].values[0], axis=1)

# sort by distance ascending
wijken = wijken.sort_values('distance')

# show the 5 most similar wijken
print(f'5 most similar wijken for {sample.wijknaam.values[0]} - {sample.gemeentenaam.values[0]}:')
wijken[1:6][['gemeentenaam', 'wijknaam'] + features + ['distance']]


# import plotly.express as px

# fig = px.scatter_3d(wijken, x='BEV_DICHTH', y='L0_altieri', z='L1_altieri', color='distance', hover_data=['gemeentenaam', 'wijknaam'])
# fig.show()

TypeError: unsupported operand type(s) for /: 'list' and 'int'